# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
#import libraries
#General
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine
#Natural Languages
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
#Machine Learning
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("disaster_table", con=engine)

In [6]:
col_names = df.columns[5:]
X = df.message.values
Y = df[col_names].values

In [7]:
print(X.shape)
print(Y.shape)

(26180,)
(26180, 35)


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    '''
    Tokenize, lemmatization and delete stopwords for Machine Learning use.
    
    Input: str
    Output: list
    '''
    text = re.sub(r'[^A-Za-z0-9]', ' ', text.lower())
    words = word_tokenize(text)
    result = [WordNetLemmatizer().lemmatize(w) for w in words 
                  if w not in stopwords.words('english')]  
    return result

In [6]:
print(X[8])
print(tokenize(X[8]))

I would like to receive the messages, thank you
['would', 'like', 'receive', 'message', 'thank']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100))),
])

In [8]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [10]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pipeline.predict(X_test)

In [12]:
print(classification_report(y_test, y_pred, target_names=col_names))

                        precision    recall  f1-score   support

               request       0.84      0.47      0.60      1466
                 offer       0.00      0.00      0.00        41
           aid_related       0.75      0.69      0.72      3549
          medical_help       0.64      0.10      0.17       688
      medical_products       0.77      0.09      0.16       411
     search_and_rescue       0.65      0.04      0.08       245
              security       0.00      0.00      0.00       170
              military       0.68      0.06      0.11       286
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.34      0.49       543
                  food       0.83      0.58      0.69       939
               shelter       0.81      0.40      0.54       762
              clothing       0.56      0.08      0.14       123
                 money       0.83      0.03      0.05       195
        missing_people       0.00      

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1

In [13]:
accuracy = (y_pred == y_test).mean()
print("Accuracy:", round(accuracy, 4))

Accuracy: 0.952


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {'vect__max_df': (0.5, 0.75, 1.0),
              'vect__ngram_range': ((1, 1), (1,2)),
              'vect__max_features': (None, 5000,10000),
              'tfidf__use_idf': (True, False)}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=3, verbose=10)

In [15]:
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [18]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:  2.8min
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed: 11.3min
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed: 14.4min
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed: 21.3min
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed: 26.3min
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed: 34.8min
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 45.5min
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed: 55.0min
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed: 63.7min
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed: 78.0min
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed: 88.0min
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 102.7min
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 114.4min
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 130.1min
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [ ]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))

In [ ]:
accuracy = (y_pred == y_test).mean()
print("Accuracy:", round(accuracy, 4))
print("\nBest Parameters:", cv.best_params_)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
#joblib.dump(cv, 'random_forest.pkl')
#joblib.dump(cv.best_estimator_, 'random_forest_best.pkl')